In [1]:
# 1. Import the WeatherPy_database.csv file. 
import pandas as pd
import matplotlib.pyplot as plt
import time
from config import maps_api_key
from config import weather_api_key
import requests
import gmaps

city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Gumdag,TM,39.21,54.59,82.62,11,36,14.92,scattered clouds
1,1,Jamestown,US,42.10,-79.24,60.44,42,100,4.61,overcast clouds
2,2,Saint-Philippe,RE,-21.36,55.77,71.19,66,55,7.25,broken clouds
3,3,Castro,BR,-24.79,-50.01,53.58,91,16,3.42,few clouds
4,4,Mar del Plata,AR,-38.00,-57.56,48.09,90,100,5.75,fog


In [2]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Declaring variables in a state that will allow the program to loop and ask again if a variable is entered incorrectly
min_temp = None
max_temp = None

# Creating a loop where a user will be brought back to input if they input incorrectly, and given a reason why
while min_temp or max_temp == None:
    try:
        min_temp = float(input("What is the minimum temperature you would like for your trip? "))
        max_temp = float(input("What is the maximum temperature you would like for your trip? "))
    except:
        print("Invalid input, please enter a floating point number\r", end = "")
        time.sleep(1.5)
    else:
        break
        
# Displays user selected temperature bounds for confirmation
print("\n\nSounds good!\n")
print(f"We'll look for destinations between {min_temp}F and {max_temp}F for you!")


What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 120


Sounds good!

We'll look for destinations between 20.0F and 120.0F for you!


In [3]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temps_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                            (city_data_df["Max Temp"] >= min_temp)]

In [4]:
# 4a. Determine if there are any empty rows.
temps_df.count()

Unnamed: 0             648
City                   648
Country                641
Latitude               648
Longitude              648
Max Temp               648
Humidity               648
Cloudiness             648
Wind Speed             648
Current Description    648
dtype: int64

In [5]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
cleaned_temps_df = temps_df.dropna()

In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = cleaned_temps_df[["City", "Country", "Max Temp", "Current Description", "Latitude", "Longitude"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Latitude,Longitude,Hotel Name
0,Gumdag,TM,82.62,scattered clouds,39.21,54.59,
1,Jamestown,US,60.44,overcast clouds,42.10,-79.24,
2,Saint-Philippe,RE,71.19,broken clouds,-21.36,55.77,
3,Castro,BR,53.58,few clouds,-24.79,-50.01,
4,Mar del Plata,AR,48.09,fog,-38.00,-57.56,
5,Los Llanos de Aridane,ES,65.95,scattered clouds,28.66,-17.92,
6,Hamilton,US,75.56,clear sky,39.18,-84.53,
7,Ushuaia,AR,47.86,broken clouds,-54.80,-68.30,
8,Jaraguá,BR,66.58,clear sky,-15.76,-49.33,
9,Longyearbyen,SJ,44.44,scattered clouds,78.22,15.64,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": maps_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, city in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = city.Latitude
    long = city.Longitude
    
    # Also adding another entry the params dict 
    params["location"] = (f"{lat},{long}")
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    top_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(top_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:  
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")      

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [20]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df[hotel_df["Hotel Name"] == ""]
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Latitude,Longitude,Hotel Name
0,Gumdag,TM,82.62,scattered clouds,39.21,54.59,HOSPEDAJE XIMENA
1,Jamestown,US,60.44,overcast clouds,42.10,-79.24,Hampton Inn & Suites Jamestown
2,Saint-Philippe,RE,71.19,broken clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
3,Castro,BR,53.58,few clouds,-24.79,-50.01,CHACARA BAILLY
4,Mar del Plata,AR,48.09,fog,-38.00,-57.56,Gran Hotel Mar del Plata


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather </dt><dd> {Current Description} and {Max Temp} °F</dd>
</dl> """

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Latitude", "Longitude"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))